In [42]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(200_000_000_000)

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics
import branca

import shared_utils
import utils

from siuba import *
import pandas as pd
import geopandas as gpd
import shapely

import datetime as dt
import time
from zoneinfo import ZoneInfo

import rt_analysis as rt
import importlib

importlib.reload(utils)
importlib.reload(rt)

import gcsfs
fs = gcsfs.GCSFileSystem()

from tqdm import tqdm_notebook
from tqdm.notebook import trange, tqdm

In [2]:
air_joined = pd.read_parquet('airtable_joined.parquet')

In [3]:
analysis_date = dt.date(2022, 2, 8) ##tuesday, new tables

In [4]:
colorscale = branca.colormap.step.RdYlGn_10.scale(vmin=0, vmax=30)
colorscale.caption = "Speed (miles per hour)"

In [5]:
# d3 = [273, 372, 221]

In [6]:
ran_operators = [int(filename.split('_')[0]) for filename in os.listdir('speedmaps') if filename[0] != '.']

In [43]:
pbar = tqdm()

0it [00:00, ?it/s]

In [45]:
not_ran = [278]

In [35]:
# ran_operators = [135, 273, 24, 127, 282, 194, 290, 301, 170, 295, 182, 300]

In [44]:
analysis_date = dt.date(2022, 2, 17) ##one-off sdmts

In [ ]:
# for agency in air_joined.calitp_itp_id.unique():
for agency in not_ran:
    not_ran_operators = []
    if agency in ran_operators:
        print(f'already ran: {agency}')
        continue
    print(f'calculating for agency: {agency}...')
    try:
        rt_day = rt.OperatorDayAnalysis(agency, analysis_date, pbar)
        rt_day.set_filter(start_time='15:00', end_time='19:00')
        # rt_day.filter_formatted = ', PM Peak, Feb 08 (Tue)' ##TODO fix hardcode
        # rt_day.agency_name = (air_joined >> filter(_.calitp_itp_id == agency)).Name.iloc[0]
        m = rt_day.segment_speed_map(how='low_speeds', colorscale=colorscale, size = [1300, 700])
        m.save(f'./speedmaps/{agency}_02_08_pm_peak.html')
        rt_day.stop_segment_speed_view.to_parquet(f'./segment_speed_views/{agency}_02_08_pm_peak.parquet') ## early step towards scaling
        delay_to_parquet = rt_day.endpoint_delay_view.copy()
        delay_to_parquet['delay_seconds'] = delay_to_parquet.delay.map(lambda x: x.seconds)
        delay_to_parquet = delay_to_parquet >> select(-_.delay)
        delay_to_parquet.to_parquet(f'./endpoint_delay_views/{agency}_02_08_pm_peak.parquet') ## early step towards scaling
        print(f'complete for agency: {agency}')
    except Exception as e:
        print(f'rt failed for agency {agency}')
        not_ran_operators += [agency]
        print(e)

calculating for agency: 278...


/home/jovyan/data-analyses/_shared_utils/shared_utils/utils.py:36: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')


found_parquet
3 scheduled trips out of 8104 have no shape, dropping
vehicle positions gdf must not be empty
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data


### Ongoing issues
* 358 Union City too many trips without shape?

In [10]:
# air_joined['url'] = air_joined.apply(lambda x:
#                     f'http://docs.calitp.org/data-analyses/rt_delay/speedmaps/{x.calitp_itp_id}_02_08_pm_peak.html',
#                                     axis = 1)

# air_joined.to_csv('linked.csv')